In [114]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from typing import Union, List, Tuple, Any, Callable, Dict
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

The following code is an expansion of the last notebook, the aim was to get a better eval metric and a potential broader model that can recommend for more books.

In [111]:
book_df = pd.read_csv('../Data/Upgraded_data/Books.csv',  sep=';', quotechar='"', header=0, encoding='latin1', error_bad_lines=False, warn_bad_lines=True)
book_ratings_df = pd.read_csv('../Data/Upgraded_data/Book-Ratings.csv', sep=';', quotechar='"', header=0, encoding='latin1', error_bad_lines=False, warn_bad_lines=True)
users_df = pd.read_csv('../Data/Upgraded_data/Users.csv', sep=';', quotechar='"', header=0, encoding='latin1', error_bad_lines=False, warn_bad_lines=True)

C:\Users\alexj\AppData\Local\Temp\ipykernel_18560\2046703773.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_df = pd.read_csv('../Data/Upgraded_data/Books.csv',  sep=';', quotechar='"', header=0, encoding='latin1', error_bad_lines=False, warn_bad_lines=True)
C:\Users\alexj\AppData\Local\Temp\ipykernel_18560\2046703773.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_df = pd.read_csv('../Data/Upgraded_data/Books.csv',  sep=';', quotechar='"', header=0, encoding='latin1', error_bad_lines=False, warn_bad_lines=True)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 

In [112]:
book_df = book_df.join(book_ratings_df.groupby('ISBN').count()['Book-Rating'], on='ISBN')
book_df = book_df.rename(columns={"Book-Rating": "Book-Rating-Count"})
book_df = book_df.join(book_ratings_df.groupby('ISBN').mean()['Book-Rating'], on='ISBN')
book_df = book_df[book_df['Year-Of-Publication'] != 'DK Publishing Inc']
book_df = book_df[book_df['Year-Of-Publication'] != 'Gallimard']
book_df['Year-Of-Publication'] = book_df['Year-Of-Publication'].astype(int)
book_df = book_df[book_df['Year-Of-Publication']>=1900]
book_df = book_df[book_df['Year-Of-Publication']<=2022]
book_df = book_df.dropna()

In [82]:
book_df.to_csv('../Data/Clean_book_data.csv')

In [58]:
book_df

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating-Count,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,1.0,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,14.0,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,3.0,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,11.0,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,1.0,7.000000
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,1.0,4.000000
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,1.0,0.000000
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,1.0,0.000000


In [108]:
def preprocess_data(df: pd.DataFrame, threshold: Union[float, int] =0.3) -> pd.DataFrame:
    """
    Performs combined preprocessing on the given DataFrame. It includes cleaning, 
    formatting, and preparing the data for the KNN model.

    Args:
    df (pd.DataFrame): The DataFrame to preprocess.
    threshold (float or int): The threshold for dropping columns with NaN values.

    Returns:
    tuple: A tuple containing the ColumnTransformer and the preprocessed DataFrame.

    """
    nan_percentage = df.isna().mean()
    df = df.drop(columns=nan_percentage[nan_percentage > threshold].index)
    features = ['Year-Of-Publication', 'Book-Rating-Count', 'Book-Rating', 'Publisher', 'Book-Author']
    df = df[features]
    column_transformer = ColumnTransformer([
        ('numerical', StandardScaler(), ['Year-Of-Publication', 'Book-Rating-Count', 'Book-Rating']),
        ('categorical', OneHotEncoder(handle_unknown='ignore'), ['Publisher', 'Book-Author']),
    ])

    return column_transformer, df

In [59]:

column_transformer, processed_df = preprocess_data(book_df[book_df['Book-Rating-Count']>19])

knn = Pipeline(steps=[('preprocessor', column_transformer),
                      ('classifier', NearestNeighbors(n_neighbors=5, metric='cosine'))])

knn.fit(processed_df)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical', StandardScaler(),
                                                  ['Year-Of-Publication',
                                                   'Book-Rating-Count',
                                                   'Book-Rating']),
                                                 ('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Publisher',
                                                   'Book-Author'])])),
                ('classifier', NearestNeighbors(metric='cosine'))])

In [60]:
with open('../Models/knn_model_20.pkl', 'wb') as file:
    pickle.dump(knn, file)

In [7]:
def make_recommendation(model: Pipeline, 
                        df: pd.DataFrame, 
                        book_title_or_isbn: str, 
                        top_n: int = 5) -> Union[List[str], str]:
    """
    Makes book recommendations based on a given book title or ISBN, excluding the input book.

    Args:
    model (Pipeline): The trained KNN pipeline model.
    df (pd.DataFrame): The DataFrame containing the book data.
    book_title_or_isbn (str): The book title or ISBN to base recommendations on.
    top_n (int): The number of recommendations to return (default is 5).

    Returns:
    Union[List[str], str]: A list of recommended book titles, excluding the input book, 
                           or an error message if the book is not found.
    """
    book = df[(df['Book-Title'].str.contains(book_title_or_isbn, na=False, case=False)) |
              (df['ISBN'] == book_title_or_isbn)]
    if book.empty:
        return "Book not found."

    index = book.index[0]
    preprocessed_book_features = model.named_steps['preprocessor'].transform(df[df.index == index])
    distances, indices = model.named_steps['classifier'].kneighbors(preprocessed_book_features, n_neighbors=top_n + 1)
    
    recommended_indices = indices[0]
    recommended_indices = recommended_indices[recommended_indices != index][:top_n]
    
    recommendations = df.iloc[recommended_indices]['Book-Title'].tolist()

    return recommendations




In [8]:

book_title_or_isbn = "There's a Bat in Bunk Five"
recommendations = make_recommendation(knn, book_df, book_title_or_isbn)
recommendations

["There's a Bat in Bunk Five",
 'Cat Ate My Gymsuit',
 'Zia',
 'Many Waters (Laurel Leaf Books)',
 'Heroes, Gods and Monsters of Greek Myths']

In [9]:
def make_isbn_recommendation(model: Pipeline, 
                             df: pd.DataFrame, 
                             book_title_or_isbn: str, 
                             top_n: int = 5) -> Union[List[str], str]:
    """
    Makes book recommendations based on a given book title or ISBN, excluding the input book.

    Args:
    model (Pipeline): The trained KNN pipeline model.
    df (pd.DataFrame): The DataFrame containing the book data.
    book_title_or_isbn (str): The book title or ISBN to base recommendations on.
    top_n (int): The number of recommendations to return.

    Returns:
    Union[List[str], str]: A list of recommended book ISBNs, excluding the input book,
                           or an error message if the book is not found.
    """
    book = df[(df['Book-Title'].str.contains(book_title_or_isbn, na=False, case=False)) |
              (df['ISBN'] == book_title_or_isbn)]
    if book.empty:
        return "Book not found."

    index = book.index[0]
    preprocessed_book_features = model.named_steps['preprocessor'].transform(df[df.index == index])
    distances, indices = model.named_steps['classifier'].kneighbors(preprocessed_book_features, n_neighbors=top_n + 1)
    
    recommended_indices = indices[0]
    recommended_indices = recommended_indices[recommended_indices != index][:top_n]
    
    recommendations = df.iloc[recommended_indices]['ISBN'].tolist()

    return recommendations



In [101]:
def calculate_diversity(recommendations: List[str]) -> float:
    """
    Calculate the diversity of the given list of recommendations.

    Arguments:
    recommendations (List[str]): A list of recommended items.

    Returns:
    float: The diversity score of the recommendations.
    """
    unique_recommendations = len(set(recommendations))
    total_recommendations = len(recommendations)
    diversity = unique_recommendations *100/ total_recommendations
    return diversity

In [102]:
def calculate_catalog_coverage(recommendations: List[str], catalog: List[str]) -> float:
    """
    Calculate the catalog coverage of the given recommendations.

    Arguments:
    recommendations (List[str]): A list of recommended items.
    catalog (List[str]): The complete list of items in the catalog.

    Returns:
    float: The catalog coverage percentage.
    """
    recommended_items = set(recommendations)
    catalog_items = set(catalog)
    coverage = len(recommended_items.intersection(catalog_items))*100 / len(catalog_items)
    return coverage

Up to here is generally exactly the same code with some minor difference in preprocessing.

In [11]:
unique_titles = book_df.drop_duplicates(subset='ISBN')['Book-Title'].tolist()


In [28]:
def get_all_recommendations(model: Any, test_df: pd.DataFrame, full_df: pd.DataFrame, top_n: int = 5) -> List[str]:
    """
    Generate a list of all recommendations for each item in the test DataFrame.

    Arguments:
    model (Any): The recommendation model to be used.
    test_df (DataFrame): The test DataFrame containing the data for which recommendations are to be made.
    full_df (DataFrame): The full DataFrame containing all data available for making recommendations.
    top_n (int): The number of top recommendations to generate for each item (default is 5).

    Returns:
    List[str]: A list of all recommendations.
    """
    all_recommendations = []
    for _, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
        book_title_or_isbn = row['isbn'] 
        recommendations = make_recommendation(model, full_df, book_title_or_isbn, top_n)
        try:
            
            if isinstance(recommendations, list):
                all_recommendations.extend(recommendations)
        except Exception as e:
            print(e)
            print(book_title_or_isbn)
        
    return all_recommendations

In [36]:
sampled_df = book_df.sample(n=2000, random_state=42)
recommendations = get_all_recommendations(knn, sampled_df, book_df)

100%|██████████| 2000/2000 [11:52<00:00,  2.81it/s]


Here I had a limit on amount I could sampe as you can see takes a while to run.

In [12]:
print(f'Diversity score: {calculate_diversity(recommendations)} %')
print(f'Catalog coverage: {calculate_catalog_coverage(recommendations, unique_titles)} %')

Diversity score: 100.0 %
Catalog coverage: 0.002105759673859942 %


So every recommendation is entirely different, this is interesting, this and the catalog coverage will probably be as a result of the size of the data set that has been created. 

In [13]:
user_reviews = book_ratings_df.groupby('User-ID')['ISBN'].apply(list).to_dict()

book_sample = book_ratings_df['ISBN'].drop_duplicates().sample(n=2000).tolist()

In [27]:
book_recommendations = {}
for book in tqdm(book_sample, desc="Generating Recommendations"):
    book_recommendations[book] = make_isbn_recommendation(knn, book_df, book)

Generating Recommendations: 100%|██████████| 2000/2000 [08:06<00:00,  4.11it/s]


In [141]:
def calculate_hit_rate(user_reviewed_books: list, recommended_books: list) -> float:
    """
    Calculates the hit rate, i.e., the proportion of recommended books that were also reviewed by the user.

    
    Args:
    user_reviewed_books (list): List of book IDs reviewed by the user.
    recommended_books (list): List of book IDs recommended for a specific book.

    Returns:
    float: Hit rate for the given user and book.
    """
    hits = sum(book in user_reviewed_books for book in recommended_books)
    return hits / len(recommended_books) if recommended_books else 0

def overall_hit_rate(users_reviews: dict, book_recommendations: dict) -> float:
    """
    Calculates the overall hit rate for a recommendation system across multiple users and books.

    Args:
    users_reviews (dict): Dictionary mapping user IDs to lists of reviewed book IDs.
    book_recommendations (dict): Dictionary mapping book IDs to lists of recommended book IDs.

    Returns:
    float: Overall hit rate of the recommendation system.
    """
    total_hit_rate = 0
    count = 0

    for user, reviewed_books in users_reviews.items():
        for book in reviewed_books:
            if book in book_recommendations:
                hit_rate = calculate_hit_rate(reviewed_books, book_recommendations[book])
                total_hit_rate += hit_rate
                count += 1

    return total_hit_rate / count if count else 0


In [32]:
overall_hit_rate_value = overall_hit_rate(user_reviews, book_recommendations)
overall_hit_rate_value

0.2539134783696163

So this is good, at least 25% of the time the recomendation is one that a person has also reviewed as well. Considering the cold start state of the problem this one of the more useful metrics from this task. Given all this lets create an app that will run this. Please feel free to run the KNN app in the terminal. Finally the rest of this notebook will be abit of sandboxing from myself to see what other models I can try, given the time constraint the evididence will be mainly empirical but the aim is to show some other options for future work.

In [7]:
users_df = users_df.dropna()

In [8]:
users_df

,User-ID,Location,Age
1,2,"stockton, california, usa",18.0
3,4,"porto, v.n.gaia, portugal",17.0
5,6,"santa monica, california, usa",61.0
9,10,"albacete, wisconsin, spain",26.0
10,11,"melbourne, victoria, australia",14.0
...,...,...,...
278848,278849,"georgetown, ontario, canada",23.0
278850,278851,"dallas, texas, usa",33.0
278851,278852,"brisbane, queensland, australia",32.0
278852,278853,"stranraer, n/a, united kingdom",17.0


In [9]:
def extract_location_components(location: str) -> str:
    """
    Extract the country component from a location string.

    This function splits the input string by ', ' and attempts to return the third
    component, which is assumed to be the country. If the input string does not
    contain enough components, it returns 'Unknown'.

    Args:
    location (str): A location string formatted as 'City, Region, Country'.

    Returns:
    str: The country component of the location or 'Unknown' if not available.
    """
    components = location.split(', ')

    country = components[2] if len(components) > 2 else 'Unknown'
    return country

users_df['Location'] = users_df.apply(
    lambda row: pd.Series(extract_location_components(row['Location'])), axis=1
)

In [10]:
location_dummies = pd.get_dummies(users_df['Location'], prefix='Loc')
location_dummies
users_df = pd.concat([users_df, location_dummies], axis=1)
users_df

,User-ID,Location,Age,Loc_,Loc_ pasig city.,Loc_&#20013;&#22269;,Loc_&#32654;&#22269;,"Loc_,",Loc_5057chadwick ct.,Loc_600 083,...,Loc_you listed stroud!),Loc_ysa,Loc_yu-song,Loc_yugoslavia,Loc_zambia,Loc_zelezny brod,Loc_zimbabwe,Loc_álava,Loc_ä¸­å?½,Loc_öð¹ú
1,2,usa,18.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,portugal,17.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,usa,61.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,spain,26.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,11,australia,14.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278848,278849,canada,23.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278850,278851,usa,33.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278851,278852,australia,32.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278852,278853,united kingdom,17.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
users_df['Age'] = (users_df['Age'] - users_df['Age'].mean()) / users_df['Age'].std()

In [12]:
user_features = users_df[['Age'] + list(location_dummies.columns)]

In [ ]:
demographic_similarity = cosine_similarity(user_features.head(20000))
demographic_similarity_df = pd.DataFrame(demographic_similarity, index=user_features.head(20000).index, columns=user_features.head(20000).index)


In [30]:
location_dummies

,Loc_,Loc_ pasig city.,Loc_&#20013;&#22269;,Loc_&#32654;&#22269;,"Loc_,",Loc_5057chadwick ct.,Loc_600 083,Loc_?ú?{,Loc_Unknown,"Loc_\""n/a\""""",...,Loc_you listed stroud!),Loc_ysa,Loc_yu-song,Loc_yugoslavia,Loc_zambia,Loc_zelezny brod,Loc_zimbabwe,Loc_álava,Loc_ä¸­å?½,Loc_öð¹ú
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278848,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278850,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278851,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
normalized_age = (30 - users_df['Age'].mean()) / users_df['Age'].std()
encoded_location = location_dummies.iloc[0]
inference_user_features = np.hstack([normalized_age, encoded_location])


In [14]:
user_similarity_scores = cosine_similarity([inference_user_features], user_features)[0]
print(user_similarity_scores)


[-0.73553397 -0.77557503  0.89189856 ... -0.18721998 -0.77557503
  0.72597536]


Cosine is perfect as removes magnitude and shows purely direction based.

In [15]:
similar_users = users_df.index[user_similarity_scores > 0.9]
similar_users

Int64Index([   171,    220,    251,    482,    539,    678,    689,    721,
               729,    825,
            ...
            278471, 278494, 278585, 278620, 278621, 278712, 278738, 278774,
            278806, 278820],
           dtype='int64', length=5671)

In [16]:
similar_users_ratings = book_ratings_df[book_ratings_df['User-ID'].isin(similar_users)]
top_books = similar_users_ratings.groupby('ISBN')['Book-Rating'].mean().sort_values(ascending=False).head(5)

In [17]:
top_books

ISBN
0152164502    10.0
04492142      10.0
0786906162    10.0
0786906456    10.0
0553381407    10.0
Name: Book-Rating, dtype: float64

In [18]:
book_df[book_df.ISBN.isin(top_books.index)]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
17577,0152164502,The Coffin Quilt: The Feud between the Hatfiel...,Ann Rinaldi,2001,Gulliver Books Paperbacks,http://images.amazon.com/images/P/0152164502.0...,http://images.amazon.com/images/P/0152164502.0...,http://images.amazon.com/images/P/0152164502.0...
37130,0786906456,Soulforge: A Novel (Dragonlance Saga),Margaret Weis,1998,TSR,http://images.amazon.com/images/P/0786906456.0...,http://images.amazon.com/images/P/0786906456.0...,http://images.amazon.com/images/P/0786906456.0...
37161,0786906162,The Dawning of a New Age (Dragonlance Dragons ...,Jean Rabe,1996,TSR,http://images.amazon.com/images/P/0786906162.0...,http://images.amazon.com/images/P/0786906162.0...,http://images.amazon.com/images/P/0786906162.0...
68600,0553381407,Toxic Parents: Overcoming Their Hurtful Legacy...,Susan Forward,2002,Bantam,http://images.amazon.com/images/P/0553381407.0...,http://images.amazon.com/images/P/0553381407.0...,http://images.amazon.com/images/P/0553381407.0...


In [64]:
def eval_similarity_model(sample_df: pd.DataFrame, 
                          user_features: np.ndarray, 
                          book_df: pd.DataFrame, 
                          book_ratings_df: pd.DataFrame, 
                          users_df: pd.DataFrame, 
                          threshold: float = 0.99) -> Tuple[float, float]:
    """
    Evaluate a similarity model for book recommendations.

    This function iterates over a sample dataframe, calculates user similarity
    scores using a cosine similarity metric, and generates book recommendations. 
    It then calculates the catalog coverage and diversity of these recommendations.

    Args:
    sample_df (pd.DataFrame): A dataframe containing the sample data.
    user_features (np.ndarray): An array of user feature vectors.
    book_df (pd.DataFrame): A dataframe containing book data.
    book_ratings_df (pd.DataFrame): A dataframe containing user book ratings.
    users_df (pd.DataFrame): A dataframe containing user data.
    threshold (float): A threshold value for similarity scores. Defaults to 0.99.

    Returns:
    Tuple[float, float]: A tuple containing the catalog coverage and diversity 
                         of the recommendations.
    """
    all_recommendations = []
    catalog = set(book_df['ISBN'])

    for _, encoded_location in tqdm(sample_df.iterrows(),
                                    total=sample_df.shape[0]):
        normalized_age = ((np.random.randint(1, 100) - 
                           users_df['Age'].mean()) / 
                           users_df['Age'].std())
  
        inference_user_features = np.hstack([normalized_age, 
                                             encoded_location.values])
        user_similarity_scores = cosine_similarity([inference_user_features], 
                                                   user_features)[0]

        similar_users = users_df.index[user_similarity_scores > threshold]
        
        if not similar_users:
            continue
        similar_users_ratings = book_ratings_df[book_ratings_df['User-ID']
                                                .isin(similar_users)]
        top_books = (similar_users_ratings.groupby('ISBN')['Book-Rating']
                     .mean().sort_values(ascending=False)
                     .head(5).index.tolist())
        
        all_recommendations.extend(top_books)

    catalog_coverage = calculate_catalog_coverage(all_recommendations, catalog)
    diversity = calculate_diversity(all_recommendations)

    return catalog_coverage, diversity

In [103]:
catalog_coverage, diversity = eval_similarity_model(location_dummies.sample(500), user_features, book_df, book_ratings_df, users_df, threshold=0.7)

100%|██████████| 500/500 [08:43<00:00,  1.05s/it]


In [104]:
print(f'Diversity score: {diversity} %')
print(f'Catalog coverage: {catalog_coverage} %')

Diversity score: 10.12 %
Catalog coverage: 0.08438974056603774 %


Not the best results this model might need alot of work.

In [66]:
def recommendation_engine(age: float, 
                          encoded_location: np.ndarray, 
                          user_features: np.ndarray, 
                          users_df: pd.DataFrame, 
                          book_ratings_df: pd.DataFrame, 
                          book_df: pd.DataFrame,
                          threshold: float = 0.99) -> Tuple[List[str], List[str]]:
    """
    Generates book recommendations based on normalized age and location encoding.

    Arguments:
    age (float): Age of the user.
    encoded_location (np.ndarray): Encoded location vector of the user.
    user_features (np.ndarray): Array of features for all users.
    users_df (pd.DataFrame): DataFrame containing user information.
    book_ratings_df (pd.DataFrame): DataFrame containing book ratings by users.
    book_df (pd.DataFrame): DataFrame containing book information.
    threshold (float): Threshold for user similarity (default 0.99).

    Returns:
    Tuple[List[str], List[str]]: A tuple containing a list of top 5 book titles and 
                                 their corresponding ISBNs recommended for the user.
    """
    normalised_age = (age - users_df['Age'].mean()) / users_df['Age'].std()
    inference_user_features = np.hstack([normalised_age, encoded_location])
    user_similarity_scores = cosine_similarity([inference_user_features], 
                                               user_features)[0]

    similar_users = users_df.index[user_similarity_scores > threshold]

    if not similar_users:
        return [], []

    similar_users_ratings = book_ratings_df[book_ratings_df['User-ID']
                                            .isin(similar_users)]
    top_isbns = (similar_users_ratings.groupby('ISBN')['Book-Rating']
                 .mean().sort_values(ascending=False)
                 .head(5).index.tolist())
    top_books = book_df[book_df['ISBN'].isin(top_isbns)]['Book-Title'].tolist()

    return top_books, top_isbns

In [67]:
def get_recommendations(age: int, 
                        country: str, 
                        locations_encoding: pd.DataFrame, 
                        recommendation_engine: Callable[[float, np.ndarray, np.ndarray, pd.DataFrame, pd.DataFrame, pd.DataFrame, float], Tuple[List[str], List[str]]], 
                        user_features: np.ndarray, 
                        users_df: pd.DataFrame, 
                        book_ratings_df: pd.DataFrame,
                        book_df: pd.DataFrame) -> Union[Tuple[List[str], List[str]], str]:
    """
    Generates book recommendations based on the user's age and country.

    Arguments:
    age (int): The age of the user.
    country (str): The country of the user.
    locations_encoding (pd.DataFrame): The table with location encodings.
    recommendation_engine (Callable): A function that takes encoded location and 
                                      age to generate recommendations.
    user_features (np.ndarray): Array of user features.
    users_df (pd.DataFrame): DataFrame containing user information.
    book_ratings_df (pd.DataFrame): DataFrame containing book ratings.
    book_df (pd.DataFrame): DataFrame containing book information.

    Returns:
    Union[Tuple[List[str], List[str]], str]: List of book recommendations and their 
                                             corresponding ISBNs if inputs are valid, 
                                             otherwise an error message.
    """

    if not 1 <= age <= 100:
        return "Error: Age must be between 1 and 100."

    encoded_vector = np.zeros(len(locations_encoding.columns))
    formatted_country = f"Loc_{country.lower()}"
    
    if formatted_country in locations_encoding.columns:
        encoded_vector[locations_encoding.columns.get_loc(formatted_country)] = 1
    else:
        return f"Error: Country '{country}' is not available in the location encodings."

    recommendations, top_isbns = recommendation_engine(age, encoded_vector, 
                                                       user_features, users_df, 
                                                       book_ratings_df, book_df)

    return recommendations, top_isbns


In [68]:
get_recommendations(21, 'usa', location_dummies, recommendation_engine, user_features, users_df, book_ratings_df)

(615,)


(['Die Brautprinzessin Actics for Market Domination',
  "Early Eight (Working Man's Mystery)",
  'Scarletti Curse (Candleglow)',
  'Marin',
  'The Gallaudet survival guide to signing'],
 ['050552421X', '0451212851', '3608953523', '0930323343', '0811800229'])

Feel free to try some ages and locations.

In [98]:
train_df, test_df = train_test_split(user_features, test_size=0.005)
test_df

,Age,Loc_,Loc_ pasig city.,Loc_&#20013;&#22269;,Loc_&#32654;&#22269;,"Loc_,",Loc_5057chadwick ct.,Loc_600 083,Loc_?ú?{,Loc_Unknown,...,Loc_you listed stroud!),Loc_ysa,Loc_yu-song,Loc_yugoslavia,Loc_zambia,Loc_zelezny brod,Loc_zimbabwe,Loc_álava,Loc_ä¸­å?½,Loc_öð¹ú
193732,-0.953101,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71473,-0.537246,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
223248,-1.022410,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249471,-0.398627,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245102,0.987557,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116663,1.749958,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197213,1.611340,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127983,-0.121390,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213848,-1.299647,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
def get_hit_count(train_features: pd.DataFrame, 
                  sample_features: np.ndarray, 
                  book_ratings_df: pd.DataFrame, 
                  user_id: Union[int, str]) -> float:
    """
    Calculate the hit count as a percentage for a recommendation system.

    This function compares the top recommended books (based on similarity scores 
    and user ratings) against the books that the user has already reviewed. It 
    returns the percentage of recommended books that the user has reviewed.

    Args:
    train_features (pd.DataFrame): A dataframe containing training feature vectors.
    sample_features (np.ndarray): An array of sample feature vectors.
    book_ratings_df (pd.DataFrame): A dataframe containing book ratings.
    user_id (Union[int, str]): The user ID for whom the hit count is calculated.

    Returns:
    float: The hit count percentage, which indicates how many of the top recommended 
           books have been reviewed by the user.
    """
    user_similarity_scores = cosine_similarity([sample_features], 
                                               train_features)[0]
    similar_users = train_features.index[user_similarity_scores > 0.99]

    if not similar_users:
        return 0.0

    similar_users_ratings = book_ratings_df[book_ratings_df['User-ID']
                                            .isin(similar_users)]
    top_isbns = (similar_users_ratings.groupby('ISBN')['Book-Rating']
                 .mean().sort_values(ascending=False)
                 .head(5).index.tolist())
    user_reviewed_books = set(book_ratings_df[book_ratings_df['User-ID'] == user_id]
                              ['ISBN'])
    hit_count = sum(isbn in user_reviewed_books for isbn in top_isbns)

    return (hit_count / len(top_isbns)) * 100 if top_isbns else 0.0


In [99]:
tqdm.pandas(desc="Processing")
hit_counts = test_df.progress_apply(lambda row: get_hit_count(train_df, row, book_ratings_df, row.name), axis=1)
print("Average Hit Count Percentage:", hit_counts.mean())

Processing: 100%|██████████| 841/841 [13:27<00:00,  1.04it/s]

Average Hit Count Percentage: 0.04756242568370987


This is not a good result as this is a percentage, however the sample size is very small (due to computational issues), also to be extra precise we could use the most recent reviews and roll backwards. As mentioned earlier one of my regrets was the computational speed of the recomendation production systesm. 

The following is a couple of other techniques that given more time I would have liked to trial, I suspect the best model would have been an ensemble of the above methods combined with the following. As these are purely theoretical ideas I have not had time to do a full analysis, however they will have functions availalbe to complete emperical analysis should you be curious to do so. Please feel free to ask me more detail on the following during the interview, I hope that the doc strings alone provide all the required explanation.

In [113]:
combined_df = pd.merge(book_ratings_df, book_df, on='ISBN')
combined_df = pd.merge(combined_df, users_df, on='User-ID')
combined_df['Age'].fillna(combined_df['Age'].median(), inplace=True)

In [132]:
combined_df = combined_df.rename(columns = {'Book-Rating_x': 'Book-Rating'})
combined_df.dtypes

User-ID                  int64
ISBN                    object
Book-Rating              int64
Book-Title              object
Book-Author             object
Year-Of-Publication      int32
Publisher               object
Image-URL-S             object
Image-URL-M             object
Image-URL-L             object
Book-Rating-Count      float64
Book-Rating_y          float64
Location                object
Age                    float64
dtype: object

In [142]:


def content_based_filtering(user_input: Dict[str, str], books_df: pd.DataFrame, 
                            top_n: int = 5) -> pd.DataFrame:
    """
    Implements content-based filtering for book recommendations.

    This function combines book attributes (author, title, publisher) into a single
    string for each book, then applies TF-IDF vectorisation. It computes cosine
    similarity between the user's preferences and book features to recommend the
    top N similar books.

    Args:
        user_input (Dict[str, str]): User's preferred author, book, and publisher.
        books_df (DataFrame): DataFrame of books with 'Book-Author', 'Book-Title',
                              and 'Publisher' columns.
        top_n (int, optional): Number of top recommendations to return. Default is 5.

    Returns:
        DataFrame: DataFrame containing top N recommended books.
    """
    books_df['combined_features'] = (
        books_df['Book-Author'] + " " + books_df['Book-Title'] + " " + books_df['Publisher']
    )
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(books_df['combined_features'])
    user_profile = (
        user_input['author'] + " " + user_input['book'] + " " + user_input['publisher']
    )
    user_profile_vec = tfidf.transform([user_profile])

    sim_scores = cosine_similarity(user_profile_vec, tfidf_matrix)
    recommended_book_indices = sim_scores[0].argsort()[:-top_n - 1:-1]
    recommended_books = books_df.iloc[recommended_book_indices]

    return recommended_books


In [147]:
from scipy.sparse import csr_matrix
def collaborative_filtering_sparse(user_id: int, combined_df: pd.DataFrame, 
                                   top_n: int = 5) -> List[str]:
    """
    Implements collaborative filtering using a sparse matrix approach.

    This function creates a user-item interaction matrix from a DataFrame and applies
    Truncated Singular Value Decomposition (SVD) for dimensionality reduction. It then
    finds users with similar preferences based on cosine similarity and recommends the
    top N books preferred by these similar users.

    Args:
        user_id (int): The user ID for whom the recommendations are to be made.
        combined_df (DataFrame): DataFrame containing user IDs, ISBNs, and ratings.
        top_n (int, optional): Number of top recommendations to return. Default is 5.

    Returns:
        List[str]: A list of ISBNs representing the top N recommended books.
    """

    combined_df['user_id_code'] = combined_df['User-ID'].astype('category').cat.codes
    combined_df['isbn_code'] = combined_df['ISBN'].astype('category').cat.codes
    user_item_matrix = csr_matrix((combined_df['Book-Rating'], (combined_df['user_id_code'], combined_df['isbn_code'])), 
                                  shape=(combined_df['User-ID'].nunique(), combined_df['ISBN'].nunique()))

    svd = TruncatedSVD(n_components=20, random_state=42)
    user_item_matrix_reduced = svd.fit_transform(user_item_matrix)
    user_idx = combined_df.loc[combined_df['User-ID'] == user_id, 'user_id_code'].iat[0]

    user_vector = user_item_matrix_reduced[user_idx]
    similarity_scores = cosine_similarity([user_vector], user_item_matrix_reduced)[0]

    similar_users_indices = np.argsort(similarity_scores)[::-1][:top_n+1]
    similar_users = combined_df.loc[combined_df['user_id_code'].isin(similar_users_indices), 'User-ID'].unique()

    recommended_books = combined_df[combined_df['User-ID'].isin(similar_users) & (combined_df['User-ID'] != user_id)]['ISBN'].unique().tolist()

    return recommended_books[:top_n]

In [148]:
def hybrid_recommendation(user_input: Dict[str, str], user_id: int, combined_df: pd.DataFrame, 
                          books_df: pd.DataFrame, top_n: int = 5) -> List[str]:
    """
    Generates book recommendations using a hybrid approach combining content-based 
    and collaborative filtering methods.

    This function first obtains recommendations based on the user's input (author, book, 
    and publisher preferences) using content-based filtering. It then fetches additional
    recommendations using collaborative filtering based on user-item interactions. The 
    final recommendation list is a combination of both, ensuring a diverse selection.

    Args:
        user_input (Dict[str, str]): User's preferred author, book, and publisher.
        user_id (int): The user ID for collaborative filtering.
        combined_df (DataFrame): DataFrame containing user IDs, ISBNs, and ratings for collaborative filtering.
        books_df (DataFrame): DataFrame of books for content-based filtering.
        top_n (int, optional): Number of top recommendations to return. Default is 5.

    Returns:
        List[str]: A list of ISBNs representing the top N recommended books.
    """
    content_recommendations = content_based_filtering(user_input, books_df)
    collaborative_recommendations = collaborative_filtering_sparse(user_id, combined_df, top_n)

    combined_recommendations = list(set(content_recommendations['ISBN'].tolist() + collaborative_recommendations))
    return combined_recommendations[:top_n]


In [153]:
recommendations = hybrid_recommendation({'author': 'J.K. Rowling', 'book': 'Harry Potter', 'publisher': 'Bloomsbury'},23, combined_df, book_df)

Content Filtering
Collaborative Filtering with Sparse Matrix


In [154]:
book_df[book_df['ISBN'].isin(recommendations)]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating-Count,Book-Rating,combined_features
305,044023722X,A Painted House,John Grisham,2001,Dell Publishing Company,http://images.amazon.com/images/P/044023722X.0...,http://images.amazon.com/images/P/044023722X.0...,http://images.amazon.com/images/P/044023722X.0...,647.0,3.187017,John Grisham A Painted House Dell Publishing C...
4578,0684872153,Angela's Ashes (MMP) : A Memoir,Frank McCourt,1999,Scribner,http://images.amazon.com/images/P/0684872153.0...,http://images.amazon.com/images/P/0684872153.0...,http://images.amazon.com/images/P/0684872153.0...,326.0,3.337423,Frank McCourt Angela's Ashes (MMP) : A Memoir ...
35086,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,2004,Scholastic Paperbacks,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,15.0,3.533333,J.K. Rowling Harry Potter and the Prisoner of ...
45490,0743464117,"Band of Brothers : E Company, 506th Regiment, ...",Stephen E. Ambrose,2002,Pocket,http://images.amazon.com/images/P/0743464117.0...,http://images.amazon.com/images/P/0743464117.0...,http://images.amazon.com/images/P/0743464117.0...,11.0,2.727273,Stephen E. Ambrose Band of Brothers : E Compan...
205143,0747561966,Harry Potter and the Philosopher's Stone,J.K. Rowling,2003,Bloomsbury,http://images.amazon.com/images/P/0747561966.0...,http://images.amazon.com/images/P/0747561966.0...,http://images.amazon.com/images/P/0747561966.0...,3.0,4.666667,J.K. Rowling Harry Potter and the Philosopher'...


In [155]:
%pip install tensorflow 

     -------------------------------------- 300.8/300.8 MB 5.8 MB/s eta 0:00:00
     ------------------------------------- 413.4/413.4 kB 13.0 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 19.6 MB/s eta 0:00:00
     ------------------------------------- 442.0/442.0 kB 27.0 MB/s eta 0:00:00
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
     ---------------------------------------- 5.5/5.5 MB 22.1 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 21.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 18.9 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached libclang-16.0.6-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ------------------------------------- 938.4/938.4 kB 29.9 MB/s eta 0:00:00


In [166]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Concatenate, Input, Dropout
from tensorflow.keras.models import Model

def build_enhanced_model(num_users: int, num_books: int, num_publishers: int, 
                         embedding_size: int, max_text_length: int, 
                         num_classes: int = 11) -> Model:
    """
    Builds an enhanced neural network model for book recommendation.

    This model uses multiple inputs including user, book, publisher, book name, and 
    publication year to recommend books. Each input type is processed with embedding 
    layers (or dense layers for text and numeric inputs). The outputs from these layers 
    are then concatenated and passed through dense layers to produce the final output.

    Args:
        num_users (int): Total number of unique users.
        num_books (int): Total number of unique books.
        num_publishers (int): Total number of unique publishers.
        embedding_size (int): Size of the embedding vector.
        max_text_length (int): Maximum length of the book name text input.
        num_classes (int, optional): Number of classes in the output layer. Default is 11.

    Returns:
        Model: A compiled Keras Model ready for training.

    Example:
        model = build_enhanced_model(1000, 5000, 300, 50, 25)
    """
    user_input = Input(shape=(1,))
    book_input = Input(shape=(1,))
    publisher_input = Input(shape=(1,))
    book_name_input = Input(shape=(max_text_length,)) 
    publication_year_input = Input(shape=(1,))

    user_embedding = Embedding(num_users, embedding_size, input_length=1)(user_input)
    book_embedding = Embedding(num_books, embedding_size, input_length=1)(book_input)
    publisher_embedding = Embedding(num_publishers, embedding_size, input_length=1)(publisher_input)

    user_vector = Flatten()(user_embedding)
    book_vector = Flatten()(book_embedding)
    publisher_vector = Flatten()(publisher_embedding)


    book_name_vector = Dense(128, activation='relu')(book_name_input)
    publication_year_vector = Dense(32, activation='relu')(publication_year_input)

    concat = Concatenate()([user_vector, book_vector, publisher_vector, book_name_vector, publication_year_vector])


    dense = Dense(256, activation='relu')(concat)
    dense = Dropout(0.5)(dense)
    output = Dense(num_classes, activation='sigmoid')(dense)  

    model = Model(inputs=[user_input, book_input, publisher_input, book_name_input, publication_year_input], outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model



In [164]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


user_ids = combined_df['User-ID'].astype('category').cat.codes
book_ids = combined_df['ISBN'].astype('category').cat.codes
publisher_ids = combined_df['Publisher'].astype('category').cat.codes


tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_df['Book-Title'])
book_titles_seq = tokenizer.texts_to_sequences(combined_df['Book-Title'])
max_text_length = max(len(seq) for seq in book_titles_seq)
book_titles_padded = pad_sequences(book_titles_seq, maxlen=max_text_length)

scaler = MinMaxScaler()
publication_years = scaler.fit_transform(np.array(combined_df['Year-Of-Publication']).reshape(-1, 1))



X = np.hstack((user_ids.values.reshape(-1, 1), 
               book_ids.values.reshape(-1, 1), 
               publisher_ids.values.reshape(-1, 1),
               book_titles_padded, 
               publication_years))

num_classes = 11  
y = to_categorical(combined_df['Book-Rating'], num_classes) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [167]:
num_users = combined_df['User-ID'].nunique()
num_books = combined_df['ISBN'].nunique()
num_publishers = combined_df['Publisher'].nunique()
embedding_size = 50

model = build_enhanced_model(num_users, num_books, num_publishers, embedding_size, max_text_length)



In [168]:

history = model.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3:3+max_text_length], X_train[:, -1]], 
                    y_train, 
                    validation_data=([X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3:3+max_text_length], X_test[:, -1]], 
                    y_test), epochs=10, batch_size=2064)


Epoch 1/10
395/395 [==============================] - 65s 160ms/step - loss: 41.6736 - accuracy: 0.5825 - val_loss: 2.0512 - val_accuracy: 0.6267
Epoch 2/10
395/395 [==============================] - 57s 145ms/step - loss: 1.8619 - accuracy: 0.6282 - val_loss: 1.7316 - val_accuracy: 0.6274
Epoch 3/10
395/395 [==============================] - 60s 151ms/step - loss: 1.6251 - accuracy: 0.6284 - val_loss: 1.5700 - val_accuracy: 0.6275
Epoch 4/10
395/395 [==============================] - 59s 150ms/step - loss: 1.5096 - accuracy: 0.6285 - val_loss: 1.4881 - val_accuracy: 0.6276
Epoch 5/10
395/395 [==============================] - 59s 149ms/step - loss: 1.4502 - accuracy: 0.6285 - val_loss: 1.4467 - val_accuracy: 0.6276
Epoch 6/10
395/395 [==============================] - 59s 149ms/step - loss: 1.4182 - accuracy: 0.6285 - val_loss: 1.4222 - val_accuracy: 0.6276
Epoch 7/10
395/395 [==============================] - 59s 149ms/step - loss: 1.4001 - accuracy: 0.6285 - val_loss: 1.4058 - val_a

In [171]:

model.evaluate([X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3:3+max_text_length], X_test[:, -1]], y_test)



6357/6357 [==============================] - 11s 2ms/step - loss: 1.3854 - accuracy: 0.6276


[1.3853849172592163, 0.6276000142097473]

In [198]:
user_id = 11676
user_books = np.array(list(set(combined_df['ISBN'])))
user_books_input = np.array([user_id] * len(user_books))

predicted_ratings = model.predict([X[:, 0], X[:, 1], X[:, 2], X[:, 3:3+max_text_length], X[:, -1]])

31784/31784 [==============================] - 47s 1ms/step


In [199]:
predicted_ratings

array([[0.8686687 , 0.07927246, 0.07997542, ..., 0.4842337 , 0.38568106,
        0.42214766],
       [0.8686687 , 0.07927246, 0.07997542, ..., 0.4842337 , 0.38568106,
        0.42214766],
       [0.8686687 , 0.07927246, 0.07997542, ..., 0.4842337 , 0.38568106,
        0.42214766],
       ...,
       [0.8686687 , 0.07927246, 0.07997542, ..., 0.4842337 , 0.38568106,
        0.42214766],
       [0.8686687 , 0.07927246, 0.07997542, ..., 0.4842337 , 0.38568106,
        0.42214766],
       [0.8686687 , 0.07927246, 0.07997542, ..., 0.4842337 , 0.38568112,
        0.42214766]], dtype=float32)

In [200]:
predicted_ratings.shape

(1017062, 11)

In [201]:
final_predictions = np.argmax(predicted_ratings, axis=1)
results = pd.DataFrame({'Predictions':final_predictions, 'Title': combined_df['Book-Title'], 'ISBN': combined_df['ISBN']})

In [202]:
results['Predictions'].value_counts()

0     1017033
8           8
2           6
3           3
9           3
7           3
1           3
6           2
10          1
Name: Predictions, dtype: int64

In [203]:
results.sort_values('Predictions', ascending=False).head(5)

,Predictions,Title,ISBN
863903,10,"Computer viruses, worms, data diddlers, killer...",031202889X
195478,9,An Elm Creek Quilts Sampler: The First Three N...,074326018X
612418,9,The Know-It-All's Guide to Life: How to Climb ...,1564146731
1002877,9,"\And Then Fuzzy Told Seve...\"": A Collection o...",0809232731
140658,8,The Wicca Book of Days: Legend and Lore for Ev...,0806516852


In [205]:
combined_df[combined_df['User-ID']==11676].sort_values('Book-Rating', ascending=False).head(5)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating-Count,Book-Rating_y,Location,Age,combined_features,user_id_code,isbn_code
23886,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,116.0,4.060345,"n/a, n/a, n/a",35.0,Nicholas Sparks The Notebook Warner Books,3566,91514
33640,11676,1841764027,10,Wellingtons Peninsula Regiments (1: The Irish ...,Mike Chappell,2003,Osprey Publishing (UK),http://images.amazon.com/images/P/1841764027.0...,http://images.amazon.com/images/P/1841764027.0...,http://images.amazon.com/images/P/1841764027.0...,3.0,6.000000,"n/a, n/a, n/a",35.0,Mike Chappell Wellingtons Peninsula Regiments ...,3566,233511
27339,11676,0451402065,10,Earth Song,Catherine Coulter,1990,Onyx Books,http://images.amazon.com/images/P/0451402065.0...,http://images.amazon.com/images/P/0451402065.0...,http://images.amazon.com/images/P/0451402065.0...,20.0,4.400000,"n/a, n/a, n/a",35.0,Catherine Coulter Earth Song Onyx Books,3566,99945
27332,11676,0425171094,10,A Village Affair,Joanna Trollope,1999,Berkley Publishing Group,http://images.amazon.com/images/P/0425171094.0...,http://images.amazon.com/images/P/0425171094.0...,http://images.amazon.com/images/P/0425171094.0...,9.0,2.666667,"n/a, n/a, n/a",35.0,Joanna Trollope A Village Affair Berkley Publi...,3566,81032
27320,11676,0452281903,10,A Man Named Dave: A Story of Triumph and Forgi...,David J. Pelzer,2000,Plume Books,http://images.amazon.com/images/P/0452281903.0...,http://images.amazon.com/images/P/0452281903.0...,http://images.amazon.com/images/P/0452281903.0...,94.0,3.223404,"n/a, n/a, n/a",35.0,David J. Pelzer A Man Named Dave: A Story of T...,3566,102761
